In [2]:
from pathlib import Path # pathlib for easy path handling
import pydicom # pydicom to handle dicom files
import matplotlib.pyplot as plt
import numpy as np
import dicom2nifti # to convert DICOM files to the NIftI format
import nibabel as nib # nibabel to handle nifti files
import os
import pydicom


In [11]:
# Code to check the spacing and dimension of each slice in an image
def check_dicom_directory(directory_path):
    dicom_files = [f for f in os.listdir(directory_path)]
    if not dicom_files:
        raise Exception("No files found in the directory.")

    uids = set()
    expected_dimensions = None
    pixel_spacing = None
    slice_thickness = None
    slice_locations = []
    dicom_file_map = {}

    for dicom_file in dicom_files:
        dicom_path = os.path.join(directory_path, dicom_file)

        try:
            ds = pydicom.dcmread(dicom_path)
        except Exception as e:
            print(f"File {dicom_file} is not a valid DICOM file: {e}")
            continue

        uids.add(ds.StudyInstanceUID)
        uids.add(ds.SeriesInstanceUID)

        if expected_dimensions is None:
            expected_dimensions = (ds.Rows, ds.Columns)
        else:
            assert (ds.Rows, ds.Columns) == expected_dimensions, "Inconsistent image dimensions."

        if pixel_spacing is None:
            pixel_spacing = ds.PixelSpacing
        else:
            assert np.allclose(ds.PixelSpacing, pixel_spacing), "Inconsistent pixel spacing."

        if slice_thickness is None:
            slice_thickness = ds.SliceThickness
            print(slice_thickness)
        else:
            assert ds.SliceThickness == slice_thickness, "Inconsistent slice thickness."
    
        slice_location = ds.ImagePositionPatient[-1]
        slice_locations.append(slice_location)
        dicom_file_map[slice_location] = dicom_file

    slice_locations = sorted(slice_locations)
    slice_diffs = np.diff(slice_locations)

    for i, diff in enumerate(slice_diffs):
        if not np.isclose(diff, slice_thickness):
            print(f"Incoherent slice spacing detected between slices:")
            print(f"File: {dicom_file_map[slice_locations[i]]} and File: {dicom_file_map[slice_locations[i+1]]}")
            print(f"Expected slice thickness: {slice_thickness}, but found difference: {diff}")

    assert len(uids) == 2, "Inconsistent UIDs found."

    print("All other checks passed successfully.")


directory_path = "../WBClara_MIM/400182218/MR 1"
check_dicom_directory(directory_path)


1.3999999761581
All other checks passed successfully.


In [5]:
import os
import shutil

# Source and destination folder paths
source_folder =  "../tmp/401314984/CT"
destination_folder =  "../tmp/401314984/CT 2"

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Loop through all files in the source folder
for filename in os.listdir(source_folder):
    # Check if the file ends with '.copy'
    if filename.endswith('.copy'):
        # Construct full file paths
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)
        
        # Move the file to the destination folder
        shutil.move(source_path, destination_path)

print("All .copy files have been moved.")



All .copy files have been moved.


In [11]:
import pydicom

def display_dicom_metadata(file_path):
    try:

        ds = pydicom.dcmread(file_path)
        
        print(f"\nMetadata for file: {file_path}\n" + "="*50)
        for elem in ds:
            try:
                # Skip large binary data elements
                if elem.VR == 'OB' or elem.VR == 'OW' or elem.VR == 'OF' or elem.VR == 'SQ' or elem.VR == 'UN' or elem.VR == 'UT':
                    continue
                
                # Display other elements
                value = str(elem.value)
                if len(value) > 80:
                    value = value[:77] + '...'  # Truncate long values
                print(f"{elem.tag} {elem.name}: {value}")
            except Exception as e:
                print(f"Error displaying element {elem.tag}: {e}")
                
        print("\n" + "="*50 + "\n")
        
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

# Example usage
directory_path = "../tmp/401314984/MR/t1_vibe_dixon_tra_p4_mbh_Gd_W_COMPOSED_Series0041/Image (0001).DCM"
display_dicom_metadata(directory_path)



Metadata for file: ../tmp/401314984/MR/t1_vibe_dixon_tra_p4_mbh_Gd_W_COMPOSED_Series0041/Image (0001).DCM
(0008, 0005) Specific Character Set: ISO_IR 100
(0008, 0008) Image Type: ['ORIGINAL', 'PRIMARY', 'M', 'NORM', 'WATER', 'DIS2D']
(0008, 0012) Instance Creation Date: 20211116
(0008, 0013) Instance Creation Time: 180009.439000
(0008, 0016) SOP Class UID: 1.2.840.10008.5.1.4.1.1.4
(0008, 0018) SOP Instance UID: 1.3.12.2.1107.5.2.41.68014.2021111618000828034718479
(0008, 0020) Study Date: 20211116
(0008, 0021) Series Date: 20211116
(0008, 0022) Acquisition Date: 20211116
(0008, 0023) Content Date: 20211116
(0008, 0030) Study Time: 165524.340000
(0008, 0031) Series Time: 180009.437000
(0008, 0032) Acquisition Time: 175642.790000
(0008, 0033) Content Time: 180009.439000
(0008, 0050) Accession Number: 07456268
(0008, 0060) Modality: MR
(0008, 0070) Manufacturer: SIEMENS
(0008, 0080) Institution Name: CHU BORDET
(0008, 0081) Institution Address: Rue Heger-Bordet 7,BRUXELLES,District,BE,10